In [3]:
#This script creates a GUI which outputs results of the system

import tkinter
from tkinter import *
import pygame
from tkinter import filedialog
import os
import numpy as np
import cv2
from astropy.wcs import WCS
import matplotlib
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import DAOStarFinder
from photutils import find_peaks
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from astropy.io import ascii
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.coordinates import FK5
from astropy.table import Column
from scipy import stats
from scipy.special import comb
import math
from PIL import Image
import time
from numba import jit
from pyquaternion import Quaternion
from Quaternion import Quat
from Quaternion import normalize
import numpy as np
from math import cos, sin, radians, degrees, atan2, sqrt
from PIL import ImageTk, Image

verified_stars = []


#******************Setting up Window*****************************
pygame.init()
root = Tk()
root.title("Star Tracker")
root.configure(background='black')

mainFrame = Frame(root, bg = "black", bd = 4)
mainFrame.grid()
childFrame = Frame(mainFrame, bg = "black")
childFrame.grid()
middleChildFrame = Frame(mainFrame, bg = "black")
middleChildFrame.grid()
youngestChildFrame = Frame(mainFrame, bg = "black")
youngestChildFrame.grid()




#****************************Functions*************************************
#Function 1
def geo_vote(event):
    
    #display
    geo_text = 'Processing...'
    label_geo.set(geo_text)
    
    fl = float(my_entry_focal.get())
    ps = float(my_entry_plate.get())
    p_pitch = (ps/206265)*fl  #pixel pitch of imager, assuming square pixel
    coeff = p_pitch/fl
    u = float(my_entry_error.get())  #average centroid uncertainty(around one pixel)
    star_guess = []
    peak = 0
    
    def binary_search(array, target, dist_min, dist_max ):
        lower = 0
        upper = len(cat_dist)
        while (lower < upper):   
            x = lower + (upper - lower) // 2
            val = cat_dist['ang_dist'][x]
            if (val > dist_min) & (val < dist_max):
                return(x)
            elif (target > val):
                if (lower == x):   
                    break        
                lower = x
            elif (target < val):
                upper = x
    
    #select image file to process
    my_filetypes = [('fits file', '.fit')]
    global file_name 
    file_name = filedialog.askopenfilename(parent=youngestChildFrame,
                                    initialdir=os.getcwd(),
                                    title="Please select a file:",
                                    filetypes=my_filetypes)
    
    hdu_list = fits.open(file_name)
    image_data = hdu_list[0].data
    
    princ_x = image_data.shape[1]/2
    princ_y = image_data.shape[0]/2

  
    #find centroids
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0, iters=5)    
    daofind = DAOStarFinder(fwhm=5.0, threshold=5.*std)    
    sources = daofind(image_data - median)    
    positions = (sources['xcentroid'], sources['ycentroid'])
    if len(sources)<5:
        threshold = median + (1 * std)
        sources = find_peaks(image_data, threshold, box_size=5)   
        positions = (sources['x_peak'], sources['y_peak'])
        peak = 1    

    #use only 15 brightest stars
    if len(sources)>15:
        if peak == 0:
            mask = sources['flux'].argsort()[-15:][::-1]
            sources = sources[mask]
        else:
            mask = sources['peak_value'].argsort()[-15:][::-1]
            sources = sources[mask]


    #table for eci coordinates
    N = len(sources)
    dtype = [('star', 'i4'), ('x', 'f8'), ('y', 'f8'), ('z', 'f8')]
    star_eci = Table(data=np.zeros(N, dtype=dtype))

    #create voting table
    votes= Table()
    star_no = 0
    rows = (comb(len(sources), 2, exact=True))*1000
    bb = Column(np.full((rows, 1), 0) )
    for n in range(len(sources)):
        votes.add_column(bb, name = star_no)
        star_no = star_no+1

    #create index array
    LIST_LENGTH = len(sources)
    index = []
    while len(index) < LIST_LENGTH:
        index.append(0)

    #covert pixel coordinates to eci coordinates
    for i in range(len(sources)):
        if peak == 1:
            star_eci[i] = (i,((sources['x_peak'][i]-princ_x)*coeff)*(math.pow(((1+((math.pow((sources['x_peak'][i]-princ_x),2))+(math.pow((sources['y_peak'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),((sources['y_peak'][i]-princ_y)*coeff)*(math.pow(((1+((math.pow((sources['x_peak'][i]-princ_x),2))+(math.pow((sources['y_peak'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),(math.pow(((1+((math.pow((sources['x_peak'][i]-princ_x),2))+(math.pow((sources['y_peak'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)))
        else:
            star_eci[i] = (sources['id'][i],((sources['xcentroid'][i]-princ_x)*coeff)*(math.pow(((1+((math.pow((sources['xcentroid'][i]-princ_x),2))+(math.pow((sources['ycentroid'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),((sources['ycentroid'][i]-princ_y)*coeff)*(math.pow(((1+((math.pow((sources['xcentroid'][i]-princ_x),2))+(math.pow((sources['ycentroid'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)),(math.pow(((1+((math.pow((sources['xcentroid'][i]-princ_x),2))+(math.pow((sources['ycentroid'][i]-princ_y),2)))*(math.pow(coeff,2)))),-0.5)))

    
    cat_dist = ascii.read("cat_dist.dat")
    
    #geometric voting and modified binary search algorithm
    for i in range(len(sources)-1):
        for j in range(i+1, len(sources)):
            star_dist = math.acos(((star_eci['x'][i])*(star_eci['x'][j]))+((star_eci['y'][i])*(star_eci['y'][j]))+((star_eci['z'][i])*(star_eci['z'][j])))
            if (star_dist != 0):
                dist_min = star_dist-u
                dist_max = star_dist+u
                k = binary_search(cat_dist['ang_dist'],star_dist, dist_min, dist_max)

                #left
                match_index= k
                while (cat_dist['ang_dist'][match_index] > dist_min):
                    votes[index[i]][i] = cat_dist['star_1_id'][match_index]
                    votes[(index[i])+1][i] = cat_dist['star_2_id'][match_index]
                    index[i] = index[i]+2
                    votes[index[j]][j] = cat_dist['star_1_id'][match_index]
                    votes[(index[j])+1][j] = cat_dist['star_2_id'][match_index]
                    index[j] = index[j]+2
                    match_index = match_index-1

                #Right
                match_index = k
                while (cat_dist['ang_dist'][match_index] < dist_max):
                    votes[index[i]][i] = cat_dist['star_1_id'][match_index]
                    votes[(index[i])+1][i] = cat_dist['star_2_id'][match_index]
                    index[i] = index[i]+2
                    votes[index[j]][j] = cat_dist['star_1_id'][match_index]
                    votes[(index[j])+1][j] = cat_dist['star_2_id'][match_index]
                    index[j] = index[j]+2
                    match_index = match_index+1

    for a in range(len(sources)):
        zeroes = rows - ((list(votes[a][:])).count(0))
        votes_subset = votes[0:zeroes]
        guess = int(stats.mode(votes_subset[a][:])[0])
        star_guess.append(guess)
        
#-------------------------------------------------------------------------------------------------------------------------
    
    #Verification
    final_ver = []
    while len(final_ver) < LIST_LENGTH:
        final_ver.append(0)
    eci_data = ascii.read("star_list.dat")

    for i in range(len(sources)-1):
        for j in range(i+1, len(sources)):
            star_dist = math.acos(((star_eci['x'][i])*(star_eci['x'][j]))+((star_eci['y'][i])*(star_eci['y'][j]))+((star_eci['z'][i])*(star_eci['z'][j])))
            dist_min = star_dist-u
            dist_max = star_dist+u
            i_index = (list(eci_data['star_id'])).index(star_guess[i])
            j_index = (list(eci_data['star_id'])).index(star_guess[j])
            act_dist = math.acos(((eci_data['x'][i_index])*(eci_data['x'][j_index]))+((eci_data['y'][i_index])*(eci_data['y'][j_index]))+((eci_data['z'][i_index])*(eci_data['z'][j_index])))
            if (act_dist > star_dist - u) & (act_dist < star_dist+u):
                final_ver[i] = final_ver[i]+1
                final_ver[j] = final_ver[j]+1            
    for i in (range(len(final_ver))):
        if final_ver[i] != max(final_ver):
            star_guess[i] = 0
        
#*************************************************Attitude Determination********************************************************
    #TRIAD algorithm

    eci_data = ascii.read("star_list.dat")

    for e in range(len(star_guess)):
        if star_guess[e] != 0:
            match1 = star_guess[e]
            index_1 = e
            break 

    for e in range (index_1+1,len(star_guess)):
        if star_guess[e] != 0:
            match2 = star_guess[e]
            index_2 = e
            break

    cat_index_1 = (list(eci_data['star_id'])).index(match1)
    cat_index_2 = (list(eci_data['star_id'])).index(match2)

    v_1_b = np.array([star_eci['x'][index_1],star_eci['y'][index_1],star_eci['z'][index_1]])
    v_1_i = np.array([eci_data['x'][cat_index_1],eci_data['y'][cat_index_1], eci_data['z'][cat_index_1]])
    v_2_b = np.array([star_eci['x'][index_2],star_eci['y'][index_2],star_eci['z'][index_2]])
    v_2_i = np.array([eci_data['x'][cat_index_2],eci_data['y'][cat_index_2], eci_data['z'][cat_index_2]])

    t_1_b = v_1_b
    t_1_i =  v_1_i

    cross_2_b = (np.cross(v_1_b, v_2_b))
    t_2_b = (cross_2_b)/(math.sqrt(math.pow(cross_2_b[0],2)+math.pow(cross_2_b[1],2)+math.pow(cross_2_b[2],2)))
    cross_2_i = (np.cross(v_1_i, v_2_i))
    t_2_i = (cross_2_i)/(math.sqrt(math.pow(cross_2_i[0],2)+math.pow(cross_2_i[1],2)+math.pow(cross_2_i[2],2)))

    t_3_b = (np.cross(t_1_b, t_2_b))
    t_3_i = (np.cross(t_1_i, t_2_i))

    r_b = np.matrix([t_1_b, t_2_b, t_3_b])
    r_b = r_b.T
    r_i = np.matrix([t_1_i, t_2_i, t_3_i])
    r_i = r_i.T

    r_matrix = r_b*r_i.T
    
    R = r_matrix
    
    #display label
    R = np.array(R)
    b = '\n'.join('\t'.join('%0.3f' %x for x in y) for y in R)
    matrix_text = 'Rotation Matrix: ' + '\n' + b
    label_matrix.set(matrix_text)

#**************************************************Euler Angles****************************************************************    
    # Checks if a matrix is a valid rotation matrix.
    def isRotationMatrix(R) :
        Rt = np.transpose(R)
        shouldBeIdentity = np.dot(Rt, R)
        I = np.identity(3, dtype = R.dtype)
        n = np.linalg.norm(I - shouldBeIdentity)
        return n < 1e-6

    # Checking 
    # Calculates rotation matrix to euler angles
    # The result is the same as MATLAB except the order
    # of the euler angles ( x and z are swapped ).
    def rotationMatrixToEulerAngles(R) :

        assert(isRotationMatrix(R))

        sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])

        singular = sy < 1e-6

        if  not singular :
            x = math.atan2(R[2,1] , R[2,2])
            x = x*(180/math.pi)
            y = math.atan2(-R[2,0], sy)
            y = y*(180/math.pi)
            z = math.atan2(R[1,0], R[0,0])
            z = z*(180/math.pi)
        else :
            x = math.atan2(-R[1,2], R[1,1])
            x = x*(180/math.pi)
            y = math.atan2(-R[2,0], sy)
            y = y*(180/math.pi)
            z = 0
            z = z*(180/math.pi)

        return np.array([x, y, z])

    angles = rotationMatrixToEulerAngles(R)
    
    #display label        
    global euler_text 
    euler_text = 'Euler Angles: ' + '\n' + 'roll: ' + str(angles[0]) + '\n' + 'pitch: ' + str(angles[1]) + '\n' + 'yaw: ' + str(angles[2])
    label_euler.set(euler_text)
    
#*******************************************************Quaternion**********************************************************
    quat = Quaternion(matrix=R)
    #display label        
    quat_text = 'Quaternion: ' + '\n' + str(quat[0]) + '\n' + str(quat[1]) + 'i' + '\n' + str(quat[2]) + 'j' + '\n' + str(quat[3]) + 'k'
    label_quat.set(quat_text)
                                                                            
    #display label        
    geo_text = 'Processing Complete'
    label_geo.set(geo_text)
    
#**************************************************Annotated Image***********************************************************    
    #plotting
    %matplotlib qt 
    font = {'family': 'serif',
        'color':  'darkgreen',
        'weight': 'normal',
        'size': 8,
        }

    for i in (range(len(final_ver))):
        if star_guess[i] == 0:
            positions_false = (sources['xcentroid'][i],sources['ycentroid'][i])
            apertures = CircularAperture(positions_false, r=4.)
            apertures.plot(color='red', lw=1.5, alpha=0.5)

    for i in (range(len(final_ver))):
        if star_guess[i] != 0:
            positions_true = (sources['xcentroid'][i], sources['ycentroid'][i])
            apertures_true = CircularAperture(positions_true, r=4.)
            apertures_true.plot(color='blue', lw=1.5, alpha=0.5)
            plt.text(sources['xcentroid'][i]-80, sources['ycentroid'][i]-50, star_guess[i], fontdict=font)
    
    ra_dec = ascii.read("ra_dec.dat")
    ra_centre_array = []
    dec_centre_array = []

    for e in range(len(star_guess)):
        if star_guess[e] != 0:
            rd_index = (list(ra_dec['star_id'])).index(star_guess[e])
            ra = ra_dec['ra'][rd_index]
            dec = ra_dec['dec'][rd_index]
            x_pos = sources[e]['xcentroid']
            y_pos = sources[e]['ycentroid']
            ra_centre = round((((x_pos - image_data.shape[1]/2)*ps)/3600 + ra),3)
            dec_centre = round((((y_pos - image_data.shape[0]/2)*ps)/3600 + dec),3)
            ra_centre_array.append(ra_centre)
            dec_centre_array.append(dec_centre)
        
    ra_centre = round((np.median(ra_centre_array)),3)
    dec_centre = round((np.median(dec_centre_array)),3)

    text = 'RA: ' + str(ra_centre) + '; Dec: ' + str(dec_centre)

    font = {'family': 'serif',
            'color':  'darkred',
            'weight': 'normal',
            'size': 10,
            }

    plt.plot(image_data.shape[1]/2, image_data.shape[0]/2, ls='none', color='darkred', marker='+', ms=10, lw=2.0)
    plt.text(image_data.shape[1]/2-(image_data.shape[1]/6.5), image_data.shape[0]/2-50, text, fontdict=font)

    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(image_data, cmap='Greys', norm=norm)
    
#********************************************************************************************************************************      
    

#Function 2 
def display_raw(event):
    %matplotlib qt 
    hdu_list = fits.open(file_name)
    image_data = hdu_list[0].data
    plt.imshow(image_data, cmap='gray')
    plt.colorbar()
    
    
#========Star Tracker Interface========

Label(childFrame, text = "Geo Star Tracker", font = ('arial', 25,), padx=15, pady=15, bg="black",
      fg="white", justify=CENTER).grid(row=0, column=0)

Label(middleChildFrame, text = "Star Identification and Attitude Estimate for FITS Images", font = ('arial', 16,), padx=15, pady=15, bg="black",
      fg="white", justify=CENTER).grid(row=0, column=0)
                                                                                                                          
perm_label = Label(youngestChildFrame, text="Please enter the following parameters and choose an image to process", font = ('arial', 12,), padx=15, pady=15, bg="black",
      fg="white", justify=CENTER)

label_geo = StringVar()
label_matrix = StringVar()
label_euler = StringVar()
label_quat = StringVar()
                                                                                                                          

depositLabel_geo = Label(youngestChildFrame, textvariable=label_geo, font = ('arial', 14,), padx=15, pady=15, bg="black", fg="green", justify=CENTER)
depositLabel_matrix = Label(youngestChildFrame, textvariable=label_matrix, font = ('arial', 12,), padx=15, pady=15, bg="black", fg="white", justify=CENTER)
depositLabel_euler = Label(youngestChildFrame, textvariable=label_euler, font = ('arial', 12,), padx=15, pady=15, bg="black", fg="white", justify=CENTER)
depositLabel_quat = Label(youngestChildFrame, textvariable=label_quat, font = ('arial', 12,), padx=15, pady=15, bg="black", fg="white", justify=CENTER)        
                                                                                                                          

#=============Displays===============
choose_file = Button(youngestChildFrame,bd=4, height = 2, width = 20, text = "Load and Process Image", bg = "white", fg = "black")
choose_file.grid(row=5, column=0)
raw_disp = Button(youngestChildFrame,bd=4, height = 2, width = 20, text = "View Raw Image", bg = "white", fg = "black")
raw_disp.grid(row=5, column=2)
perm_label.grid(row=0, column=1)
my_label_fl = Label(youngestChildFrame, text = "focal length", bg="black", fg="white" )
my_label_fl.grid(row = 1, column = 0)
my_entry_focal = Entry(youngestChildFrame)
my_entry_focal.grid(row = 1, column = 1)
unit_label_fl = Label(youngestChildFrame, text = "mm", bg="black", fg="white" )
unit_label_fl.grid(row = 1, column = 2)
my_label_ps = Label(youngestChildFrame, text = " plate scale ",bg="black", fg="white" )
my_label_ps.grid(row = 2, column = 0)
my_entry_plate = Entry(youngestChildFrame)
my_entry_plate.grid(row = 2, column = 1)
unit_label_ps = Label(youngestChildFrame, text = "arcsecond/pixel",bg="black", fg="white" )
unit_label_ps.grid(row = 2, column = 2)
my_label_error = Label(youngestChildFrame, text = "estimated angular error",bg="black", fg="white" )
my_label_error.grid(row = 3, column = 0)
my_entry_error = Entry(youngestChildFrame)
my_entry_error.grid(row = 3, column = 1)
unit_label_error = Label(youngestChildFrame, text = "radians",bg="black", fg="white" )
unit_label_error.grid(row = 3, column = 2)
unit_label_error = Label(youngestChildFrame, text = " ",bg="black", fg="white" )
unit_label_error.grid(row = 4, column = 0)

depositLabel_geo.grid(row=6, column=0)
depositLabel_matrix.grid(row=7, column=0)
depositLabel_euler.grid(row=7, column=1)
depositLabel_quat.grid(row=7, column=2)


choose_file.bind('<ButtonPress-1>',geo_vote)
raw_disp.bind('<ButtonPress-1>', display_raw)

img = ImageTk.PhotoImage(Image.open("gui_back.png"), master=root )
panel = Label(youngestChildFrame, image = img)
panel.grid(row=8, column=1)

root.mainloop()